In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.stats import gaussian_kde
from collections import Counter

from pymatgen.core import Structure, Lattice
from pymatgen.io.vasp import Poscar

from polyhedral_analysis.configuration import Configuration
from polyhedral_analysis.polyhedra_recipe import PolyhedraRecipe
from polyhedral_analysis.octahedral_analysis import opposite_vertex_pairs, isomer_is_cis

from bsym.interface.pymatgen import unique_structure_substitutions

from vasppy.calculation import import_calculations_from_file

from tqdm import tqdm

%matplotlib inline
%config InlineBackend.figure_format='retina'

from figure_formatting import figure_formatting as ff
ff.formatting['axes.linewidth'] = 0.5
ff.formatting['lines.linewidth'] = 1.0
ff.set_formatting()

colors = {'blue': '#264653',
          'bluegreen': '#287271',
          'green': '#2A9D8F',
          'yellow': '#E9C46A',
          'light orange': '#F4A261',
          'dark orange': '#E76F51'}

In [2]:
recipe = PolyhedraRecipe(method='distance cutoff', 
                         coordination_cutoff=3.0, 
                         central_atoms='Ti',
                         vertex_atoms=['O','F'])

In [3]:
class Config():
    
    def __init__(self,
                 structure: Structure,
                 config_id: int,
                 energy: float,
                 degeneracy: int) -> None:
        self.structure = structure
        self.config_id = config_id
        self.energy = energy
        self.degeneracy = degeneracy
        self.config = Configuration(structure=structure, recipes=[recipe])
        
    def n_non_f4o2(self) -> int:
        count = 0
        for p in self.config.polyhedra:
            coord_count = Counter([v.label for v in p.vertices])
            if coord_count['F'] != 4:
                count += 1
        return count
    
    def n_collinear_oxygen(self) -> int:
        count = 0
        for p in self.config.polyhedra:
            vertex_pairs = opposite_vertex_pairs(p, check=False)
            for vp in vertex_pairs:
                if set([vp[0].label, vp[1].label]) == {'O'}:
                    count += 1
        return count
    
    @property
    def n_cis_f4o2(self) -> int:
        count = 0
        for p in self.config.polyhedra:
            coord_count = Counter([v.label for v in p.vertices])
            if (coord_count['F'] == 4) and isomer_is_cis(polyhedron=p, check=False):
                count += 1
        return count
    
    def bond_lengths(self) -> list[float]:
        distances = {'O': [], 'F': []}
        for p in self.config.polyhedra:
            for d, l in p.vertex_distances_and_labels():
                distances[l].append( d )
        return distances

In [4]:
data = pd.read_csv('../Data/energies.out',
                   delim_whitespace=True,
                   names=['config_id', 'ce_energy', 'dft_energy'],
                   na_values="None")
data.sort_values('config_id',
                 inplace=True)
e_min_dft_222_per_fu = data.dft_energy.min()/(2*2*2)
data.ce_energy -= data.ce_energy.min()
data.dft_energy -= data.dft_energy.min()
data

,config_id,ce_energy,dft_energy
300,0,2.21448,2.068939
2583,1,2.66760,2.529091
2641,2,2.17216,1.873055
1846,3,4.29128,4.416626
254,4,3.02408,3.449367
...,...,...,...
2616,2659,3.80192,NaN
1528,2660,0.43616,NaN
906,2661,0.66352,NaN
2547,2662,0.34864,NaN


In [5]:
poscar_dir = '../Data/poscars'

# dft_data_ids = data[data.dft_energy.notna()].config_id.values

configs = []
for i in tqdm(range(2664)):
    structure = Poscar.from_file(f'{poscar_dir}/config_{i:04d}.poscar').structure
    configs.append(Config(structure=structure,
                          config_id=i,
                          energy=data.iloc[i].ce_energy,
                          degeneracy=None))

100%|███████████████████████████████████████████████████████████████████████████████████████| 2664/2664 [00:08<00:00, 300.28it/s]


In [6]:
all_cis_f4o2 = [c.config_id for c in tqdm(configs) if c.n_cis_f4o2 == 8]

100%|███████████████████████████████████████████████████████████████████████████████████████| 2664/2664 [00:14<00:00, 180.26it/s]


In [7]:
energies_222 = data[data.config_id.isin(all_cis_f4o2)].dft_energy.values / (2*2*2)
energies_222 = energies_222[~np.isnan(energies_222)]

In [8]:
ga_444_calculations = import_calculations_from_file('../Data/vasp_summary_444.yaml')

e_444 = np.array([ga_444_calculations[t].energy for t in ["TiOF2 4x4x4 config 1",
                                                          "TiOF2 4x4x4 config 2",
                                                          "TiOF2 4x4x4 config 3",
                                                          "TiOF2 4x4x4 config 4"]]) / (4*4*4) - e_min_dft_222_per_fu

In [9]:
offo_calculations = import_calculations_from_file('../Data/vasp_summary_offo.yaml')

offo_energies = np.array([c.energy for c in offo_calculations.values()])

e_333 = offo_energies / (3*3*3) - e_min_dft_222_per_fu

In [10]:
from scipy.spatial.distance import cdist

def min_distances(arr: np.ndarray) -> np.ndarray:
    """Calculate the minimum distance from each point in a 1D
    array to its nearest neighbour in the same array
    
    Args:
        arr (np.ndarray): The input 1D array.
        
    Returns:
        np.ndarray
        
    """ 
    distances = cdist(arr.reshape(-1, 1), arr.reshape(-1, 1))
    np.fill_diagonal(distances, np.inf)
    min_dists = np.min(distances, axis=1)
    return min_dists.flatten()

In [ ]:
e_222 = energies_222[~np.isnan(energies_222)]
axes = plt.subplot(111)
kde_vscale = 0.001
baseline_offset = 0.30
kde_threshold = 1
e_min = -0.025
e_max = +0.025
e_range = np.linspace(e_min, e_max, 500)
bandwidth = 0.0012
jitter_proximity = 0.001
jitter_scale = 0.25

pcolors = {0: colors['dark orange'],
           1: colors['light orange'],
           2: colors['green']}

v_scale = {0: 0.42,
           1: 0.58,
           2: 1.7}

for n, data in enumerate([e_222, e_333, e_444]):
    to_jitter = min_distances(data) < jitter_proximity
    jitter = (np.random.random(size=len(data))*jitter_scale-jitter_scale/2) * to_jitter
    axes.plot(data, np.full_like(data, n)*2 + jitter,
             'o',
             color=pcolors[n],
             markersize=3,
             alpha=0.7, markeredgecolor=None, markeredgewidth=0.0)
    kde = gaussian_kde(dataset=data,
                       bw_method=bandwidth / data.std(ddof=1))
    baseline = n*2 + baseline_offset
    x_loc = np.where(kde(e_range) > 0)
    axes.fill_between(e_range, kde(e_range)*kde_vscale*data.size*v_scale[n] + baseline, baseline,
                      color=pcolors[n],
                      where=kde(e_range)>kde_threshold, interpolate=False,
                      linewidth=0.5, alpha=1, edgecolor=None)
    
plt.vlines(np.mean(e_444), 5.3, 5.65, color=pcolors[2], linewidth=0.75)
plt.vlines(np.mean(e_333), 3.3, 3.65, color=pcolors[1], linewidth=0.75)
plt.vlines(np.mean(e_222), 1.3, 1.65, color=pcolors[0], linewidth=0.75)

def error_bars(data):
    return np.percentile(data, 15.87), np.percentile(data, 84.13)

plt.hlines(5.475, *error_bars(e_444), color=pcolors[2], linewidth=0.75, alpha=1) 
plt.vlines(error_bars(e_444), 5.35, 5.6, color=pcolors[2], linewidth=0.75, alpha=1)

plt.hlines(3.475, *error_bars(e_333), color=pcolors[1], linewidth=0.75, alpha=1)
plt.vlines(error_bars(e_333), 3.35, 3.6, color=pcolors[1], linewidth=0.75, alpha=1)

plt.hlines(1.475, *error_bars(e_222), color=pcolors[0], linewidth=0.75, alpha=1) 
plt.vlines(error_bars(e_222), 1.35, 1.6, color=pcolors[0], linewidth=0.75, alpha=1)

axes.set_yticks([0.075, 2.075, 4.075])
axes.set_yticklabels([r'$2\times2\times2$ all-cis [OFOF]',
                      r'$3\times3\times3$ [OFFO]',
                      r'$4\times4\times4$ GA [OFOF] + [OFFF]'])
# plt.ylabel('n(O–Ti–O)')
plt.xlabel('relative energy / eV per f.u.')
axes.spines[['right', 'top', 'left']].set_visible(False)
plt.hlines([0.0, 2.0, 4.0], e_min, e_max, color='grey', alpha=0.1)
plt.xlim([e_min, e_max])
axes.tick_params(axis='y', which='both',length=0)
plt.savefig('../Figures/unit_cell_size_comparison.pdf', dpi=300)
plt.show()
